# Importing libraries

In [3]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Defining list with file names

In [4]:
files = [
    "EM2015 - szkoły (aktualizacja 09.2021)",
    "EM2015 - szkoły (aktualizacja 09.2022)",
    "EM2015 - szkoły (aktualizacja 09.2023)",
    "EM2023 - szkoły (aktualizacja 09.2023)"
]

# Combining columns with correlated exams

In [3]:
def extractExams(file: str):
    with open(file, "r", encoding="utf-8-sig") as f:
        head = f.readline()
    head = head.split(";")
    head = [element for element in head if element and not element in ["dla całego egzaminu dojrzałości", "\n"]]
    head = [element.rstrip(" (E)").rstrip(" (M)") for element in head]
    return head


for file in files:
    df = pd.read_csv(f"data/matura/{file} - raw.csv", sep=";")
    columns = df.columns.to_list()
    exames = extractExams(f"data/matura/{file} - columns.csv")
    newColumns = []
    pos = 0

    while columns[pos].split(".")[0] != "* liczba zdających":
        column = columns[pos]
        column = column.lstrip("*").lstrip(" ")
        newColumns.append(column)
        pos += 1

    for exam in exames:
        column = columns[pos]
        column = column.split(".")[0].lstrip("*").strip(" ")
        column = f"{exam} | {column}"
        newColumns.append(column)
        pos += 1

        while columns[pos].split(".")[0] != "* liczba zdających":
                column = columns[pos]
                column = column.split(".")[0].lstrip("*").strip(" ")
                column = f"{exam} | {column}"
                newColumns.append(column)
                pos += 1

                if pos >= len(columns):
                     break

    df.columns = newColumns
    df.to_csv(f"data/matura/{file}.csv", index=False, sep=";")

# Combining repeating columns

In [86]:
def combineCols(df: pd.DataFrame, col1: str, col2: str):
    """
    col1 have higher priority
    """
    df[df[col1].isnull()][col1] = df[df[col1].isnull()][col2]     
    df.drop([col2], axis=1, inplace=True)

### EM2015 - 2021

In [5]:
df = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2021).csv", sep=";")
columns = []
for col in df.columns.to_list():
    if ".1" in col:
        columns.append(col)

columns

[]

### EM2015 - 2022

In [6]:
df = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2022).csv", sep=";")
columns = []
for col in df.columns.to_list():
    if ".1" in col:
        columns.append(col)

In [7]:
for col in columns:
    combineCols(df, col.rstrip(".1"), col)

In [8]:
df.to_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2022).csv", sep=";", index=False)

### EM2015 - 2023

In [9]:
df = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2023).csv", sep=";")
columns = []
for col in df.columns.to_list():
    if ".1" in col:
        columns.append(col)

In [10]:
for col in columns:
    combineCols(df, col.rstrip(".1"), col)

In [35]:
df.to_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2023).csv", sep=";", index=False)

### EM2023 - 2023

In [36]:
df = pd.read_csv("data/matura/EM2023 - szkoły (aktualizacja 09.2023).csv", sep=";")
columns = []
for col in df.columns.to_list():
    if ".1" in col:
        columns.append(col)

In [37]:
for col in columns:
    combineCols(df, col.rstrip(".1"), col)

In [38]:
df.to_csv("data/matura/EM2023 - szkoły (aktualizacja 09.2023).csv", sep=";", index=False)

# Combining datasets

In [193]:
df1 = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2021).csv", sep=";")
df1['rok'] = 2021
df1['formuła'] = "EM2015"
df1.columns = [col.strip() for col in df1.columns]

df2 = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2022).csv", sep=";")
df2['rok'] = 2022
df2['formuła'] = "EM2015"
df2.columns = [col.strip() for col in df2.columns]

df3 = pd.read_csv("data/matura/EM2015 - szkoły (aktualizacja 09.2023).csv", sep=";")
df3['rok'] = 2023
df3['formuła'] = "EM2015"
df3.columns = [col.strip() for col in df3.columns]

df4 = pd.read_csv("data/matura/EM2023 - szkoły (aktualizacja 09.2023).csv", sep=";")
df4['rok'] = 2023
df4['formuła'] = "EM2023"
df4.columns = [col.strip() for col in df4.columns]

df = pd.concat([df1, df2, df3, df4])
df.reset_index(drop=True, inplace=True)

columns = df.columns.to_list()
columns = [col.lower() for col in columns]
df.columns = columns

### Converting colum names

In [194]:
def convertColumns(columns: list):
    # lowercase
    columns = [col.lower() for col in columns]

    # remove polish diacritical characters
    converter = {polish: normal for polish, normal in zip(list("ąęćżźśńłó"), list("aeczzsnlo"))}

    for i in range(len(columns)):
        newCol = ""
        for char in columns[i]:
            if char in converter.keys():
                newCol += converter[char]
            else:
                newCol += char
        columns[i] = newCol

    # remove unnecessary signs
    columns = ["_".join(col.split()) for col in columns]
    columns = [col.replace("-", "_").replace("___", "_") for col in columns]
    columns = [col.replace("_(%)", "") for col in columns]
    columns = [col.replace(",", "") for col in columns]
    columns = [col.replace("_|_", "_") for col in columns]
    columns = [col.replace("/finalistow", "") for col in columns]
    columns = [col.replace("mniejszosci_narodowej_", "") for col in columns]
    
    for i in range(len(columns)):
        if columns[i] == "liczba_zdajacych_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych":
            columns[i] = "liczba_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych"

    return columns

In [195]:
df.columns = convertColumns(df.columns.to_list())
df.to_csv("data/matura/matura_szkoly.csv", index=False, sep=";")

### Combining duplicated columns

In [196]:
df = pd.read_csv("data/matura/matura_szkoly.csv", sep=";")

columns = []
for col in df.columns.to_list():
    if ".1" in col or ".2" in col:
        columns.append(col)

for col in columns:
    combineCols(df, col.rstrip(".1").rstrip(".2"), col)

### Replace "," sign to "."

In [177]:
toAvoid = [
    "id_oke",
    "wojewodztwo_nazwa",
    "powiat_nazwa",
    "gmina_nazwa",
    "typ_gminy",
    "kod_teryt_gminy",
    "rspo_szkoly",
    "nazwa_szkoly",
    "miejscowosc",
    "ulica_nr",
    "typ_placowki",
    "rodzaj_placowki",
    "czy_publiczna",
    "formula",
    "rok",
    "otrzymali_swiadectwo_dojrzalosci_liczba",
    "liczba_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych",
    "identyfikator_szkoly"
]

In [178]:
def list_difference(list1, list2):
    return [item for item in list1 if item not in list2]

In [197]:
columns = list_difference(df.columns.to_list(), toAvoid)

for col in columns:
    if df[col].dtype == np.dtypes.ObjectDType:
        df[col] = df[col].str.replace(",", ".")

### Convert columns data type

In [198]:
convert = {
    "liczba_zdajacych": "int64",
    "liczba_laureatow": "int64",
    "zdawalnosc": "float64",
    "sredni_wynik": "float64",
    "odchylenie_standardowe": "float64",
    "mediana": "float64",
    "modalna": "float64"
}

for col in columns:
    for end in convert.keys():
        if end in col:
            if convert[end] == "int64":
                df[col].fillna(-1, inplace=True)
                df[col] = df[col].astype(convert[end])
            elif convert[end] == "float64":
                df[col] = df[col].astype(convert[end])
            break

In [199]:
df['otrzymali_swiadectwo_dojrzalosci_liczba'].fillna(-1, inplace=True)
df['otrzymali_swiadectwo_dojrzalosci_liczba'] = df['otrzymali_swiadectwo_dojrzalosci_liczba'].astype("int64")

In [200]:
df['liczba_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych'].fillna(-1, inplace=True)
df['liczba_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych'] = df['liczba_ktorzy_przystapili_do_wszystkich_egzaminow_wymaganych'].astype("int64")

# Set primary key

In [ ]:
df.reset_index(drop=True, inplace=True)
df.reset_index(inplace=True)
df.rename(columns={"index": "row_id"}, inplace=True)

# Save

In [205]:
df.to_csv("data/matura/matura_szkoly.csv", index=False, sep=";")